### This is a test script to understand how spectra.csv and matrix.csv work (generated by gzoltar)

In [1]:
import os
import pandas as pd

sample_folder = '/Volumes/spear_1TB/T-Evos_dc_out/spectrum/jfreechart/c7921a8d2369a1c7a8c42a4b840c31ea6c5d01b0'

data_dir = '/Volumes/spear_1TB/T-Evos_dc_out/'
spectra = pd.read_csv(os.path.join(sample_folder, 'spectra.csv'), usecols=['name'], sep=' ')
matrix = pd.read_csv(os.path.join(sample_folder, 'matrix.txt'), names=['test_matrix'])
tests = pd.read_csv(os.path.join(sample_folder, 'tests.csv'), usecols=['name','outcome','runtime','stacktrace'])

In [10]:
import re
from pathlib import Path

matrix['test_matrix'].iloc[0]
#com.alibaba.json.bvt.issue_3300.Issue3397#test_for_issue
target = 'com.alibaba.json.bvt.issue_3300.Issue3397#test_for_issue'
df_spectra = trace_coverage(target, spectra, matrix, tests)
display(df_spectra)
failed_tests = list(tests[tests['outcome']=='FAIL']['name'])

,package,class,method,parameter,line
0,com.alibaba.fastjson.parser,ParserConfig,getGlobalInstance,,167
1,com.alibaba.fastjson.parser,ParserConfig,getDeserializer,java.lang.reflect.Type,587
2,com.alibaba.fastjson.parser,ParserConfig,getDeserializer,java.lang.reflect.Type,588
3,com.alibaba.fastjson.parser,ParserConfig,getDeserializer,java.lang.reflect.Type,592
4,com.alibaba.fastjson.parser,ParserConfig,getDeserializer,java.lang.reflect.Type,593
...,...,...,...,...,...
1333,com.alibaba.fastjson.parser.deserializer,Jdk8DateCodec,write,com.alibaba.fastjson.serializer.SerializeWrite...,586
1334,com.alibaba.fastjson.parser.deserializer,Jdk8DateCodec,write,com.alibaba.fastjson.serializer.SerializeWrite...,589
1335,com.alibaba.fastjson.parser.deserializer,Jdk8DateCodec,write,com.alibaba.fastjson.serializer.SerializeWrite...,592
1336,com.alibaba.fastjson.parser.deserializer,Jdk8DateCodec,write,com.alibaba.fastjson.serializer.SerializeWrite...,593


In [8]:
import re
import os
import pandas as pd
from pathlib import Path

sample_folder = '/Volumes/spear_1TB/T-Evos_dc_out/spectrum/jfreechart/c7921a8d2369a1c7a8c42a4b840c31ea6c5d01b0^'

data_dir = '/Volumes/spear_1TB/T-Evos_dc_out/'
spectra = pd.read_csv(os.path.join(sample_folder, 'spectra.csv'), usecols=['name'], sep=' ')
matrix = pd.read_csv(os.path.join(sample_folder, 'matrix.txt'), names=['test_matrix'])
tests = pd.read_csv(os.path.join(sample_folder, 'tests.csv'), usecols=['name','outcome','runtime','stacktrace'])

for target in tests['name']:
    df_spectra = trace_coverage(target, spectra, matrix, tests)
    print(target, len(df_spectra))
    df_spectra.to_csv(os.path.join('/Volumes/spear_1TB/defects4j/d4j_induce/Jfreechart/1/prior', target+'.csv'))
    

org.jfree.chart.StackedBarChartTest#testDrawWithNullInfo 2768
org.jfree.chart.StackedBarChartTest#testSetSeriesToolTipGenerator 1096
org.jfree.chart.StackedBarChartTest#testSetSeriesURLGenerator 1101
org.jfree.chart.StackedBarChartTest#testReplaceDataset 1107
org.jfree.chart.imagemap.OverLIBToolTipTagFragmentGeneratorTest#testGenerateURLFragment 18
org.jfree.chart.imagemap.DynamicDriveToolTipTagFragmentGeneratorTest#testGenerateURLFragment 18
org.jfree.chart.imagemap.ImageMapUtilsTest#testGetImageMap 91
org.jfree.chart.imagemap.ImageMapUtilsTest#testHTMLEscape 21
org.jfree.chart.imagemap.ImageMapUtilsTest#testJavascriptEscape 15
org.jfree.chart.imagemap.ImageMapUtilsTest#testGetImageMapIllegalArgumentException 5
org.jfree.chart.imagemap.ImageMapUtilsTest#testGetImageMapIllegalArgumentException_2 10
org.jfree.chart.imagemap.StandardURLTagFragmentGeneratorTest#testGenerateURLFragment 3
org.jfree.chart.imagemap.StandardToolTipTagFragmentGeneratorTest#testGenerateURLFragment 19
org.jfree.c

In [4]:
def trace_coverage(target, spectra, matrix, tests):

    spectra_list = list(spectra['name'])

    # a_test_matrix e.g., ['0', '0', '1']
    index = tests[tests['name']==target].index.item()
    a_test_matrix = matrix['test_matrix'].iloc[index]
    a_test_matrix = a_test_matrix.split(' ')

    # regex the spectra data
    df_spectra = []
    coverage_index = [i for i, binary in enumerate(a_test_matrix) if binary == '1']
    regex = '(?P<package>.*)\$(?P<class>.*)\#(?P<method>.*)\((?P<parameter>.*)\)\:(?P<line>.*)'
    for i in coverage_index:
        match = re.search(regex, spectra_list[i])
        if match:
            df_spectra.append(match.groupdict())
        else:
            print('[error] missed a coverage', spectra_list[i], i)

    df_spectra = pd.DataFrame(df_spectra)
    return df_spectra